In [ ]:
%pylab inline

plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['font.size'] = 30


colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3','#a6d854','#ffd92f']

Populating the interactive namespace from numpy and matplotlib


In [ ]:
import numpy as np
from scipy import sparse
from sklearn.preprocessing import LabelBinarizer

from lightning.impl.sag import SAGAClassifier, get_auto_step_size
from lightning.impl.svrg import SVRGClassifier
from lightning.impl.sgd_fast import Log

# from sklearn.datasets import fetch_20newsgroups_vectorized
# data = fetch_20newsgroups_vectorized()
# X = data.data
# y = data.target.copy()
# # binarize
# y[data.target <  data.target.mean()] = -1
# y[data.target >= data.target.mean()] = -1


from sklearn.datasets import load_svmlight_file
X, y = load_svmlight_file('/Users/fabianpedregosa/data/async_saga/covtype.libsvm.binary.scale')

alpha = 1.0 / X.shape[0]
step_size = get_auto_step_size(X, alpha, 'log', None)
max_iter = 20
# step sizes are choosen as the ones that yield the fastest decrease

from datetime import datetime
coefs_saga_adaptive = [np.zeros(X.shape[1])]
start = datetime.now()
times_saga_adaptive = [0.]
logloss = Log()
def callback_adaptive(clf):
    coefs_saga_adaptive.append(
        clf.coef_scale_[0] * clf.coef_.ravel().copy())
    times_saga_adaptive.append((datetime.now() - start).total_seconds())

def compute_loss(coefs):
    loss = 0.
    for i in range(X.shape[0]):
        pred = X[i].dot(coefs)
        loss += logloss.loss(pred, y[i])
    return loss / X.shape[0] + 0.5 * alpha * coefs.dot(coefs)

clf_saga = SAGAClassifier(
    loss='log', callback=callback_adaptive, eta='adaptive', max_iter=max_iter, tol=1e-132,
    alpha=alpha, random_state=0)
clf_saga.fit(X, y)
loss_saga_adaptive = np.array([compute_loss(w) for w in coefs_saga_adaptive])
print('Done SAGAClassifier')


coefs_saga = [np.zeros(X.shape[1])]
start = datetime.now()
times_saga = [0.]
def callback(clf):
    coefs_saga.append(
        clf.coef_scale_[0] * clf.coef_.ravel().copy())
    times_saga.append((datetime.now() - start).total_seconds())


clf_saga = SAGAClassifier(
    loss='log', callback=callback, eta=step_size, max_iter=max_iter, tol=1e-132,
    alpha=alpha, random_state=0)
clf_saga.fit(X, y)
loss_saga = np.array([compute_loss(w) for w in coefs_saga])
print('Done SAGAClassifier')

In [ ]:
loss_saga_adaptive, loss_saga

In [ ]:
fmin = 0.0847302847536688
# fmin = 0.20327841000409613
lw=6
plt.plot(times_saga, np.array(loss_saga) - fmin, color=colors[0], lw=lw, label='SAGA')
plt.plot(times_saga_adaptive, np.array(loss_saga_adaptive) - fmin, color=colors[1], lw=lw, label='SAGA adaptive')
plt.yscale('log')
plt.grid()
plt.xlim((0, 20.5))
plt.ylim((1e-5, None))
plt.legend()
plt.xlabel('Time (in seconds)')
plt.ylabel('Function minus optimum')
plt.gcf().subplots_adjust(left=0.15)
# plt.savefig('rcv1_comparison.png')
plt.show()

In [ ]:
import lightning

In [ ]:
lightning??